## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-06-06-03-27 +0000,nyt,"Myra MacPherson, 91, Dies; Wrote Wrenchingly A...",https://www.nytimes.com/2026/02/05/us/politics...
1,2026-02-06-06-00-40 +0000,nypost,Former Little Caesars employee arrested after ...,https://nypost.com/2026/02/06/us-news/former-l...
2,2026-02-06-05-59-00 +0000,wsj,U.S. and Iran Gather for High Stakes Nuclear T...,https://www.wsj.com/world/middle-east/u-s-and-...
3,2026-02-06-05-49-00 +0000,wsj,Toyota Motor CEO to Step Down as Profit Slumps,https://www.wsj.com/business/autos/toyota-moto...
4,2026-02-06-05-48-31 +0000,nypost,Schumer rejected Trump’s offer to unfreeze $16...,https://nypost.com/2026/02/06/us-news/schumer-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2506/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,28
299,epstein,21
99,ai,14
130,new,13
22,talks,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
116,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...,91
125,2026-02-05-22-22-03 +0000,nypost,Hillary Clinton demands public hearing on Epst...,https://nypost.com/2026/02/05/us-news/hillary-...,71
53,2026-02-06-01-19-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,66
24,2026-02-06-03-39-02 +0000,nypost,Top Senator calls for investigation into Mayor...,https://nypost.com/2026/02/05/us-news/senator-...,60
224,2026-02-05-17-50-42 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...,59


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
116,91,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...
24,60,2026-02-06-03-39-02 +0000,nypost,Top Senator calls for investigation into Mayor...,https://nypost.com/2026/02/05/us-news/senator-...
224,59,2026-02-05-17-50-42 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...
95,48,2026-02-05-23-29-46 +0000,nypost,"Russia, Ukraine exchange 300 prisoners as US p...",https://nypost.com/2026/02/05/world-news/russi...
19,47,2026-02-06-04-26-26 +0000,nypost,AOC touts money for her NYC district in fundin...,https://nypost.com/2026/02/05/us-news/aoc-tout...
241,43,2026-02-05-17-15-17 +0000,nypost,"Stocks plunge on AI spending fears, unemployme...",https://nypost.com/2026/02/05/business/stocks-...
274,40,2026-02-05-15-27-00 +0000,wsj,The Bank of England and ECB kept interest rate...,https://www.wsj.com/economy/central-banking/ba...
108,37,2026-02-05-22-55-54 +0000,nyt,Los Angeles Civic Leader Austin Beutner Drops ...,https://www.nytimes.com/2026/02/05/us/los-ange...
93,34,2026-02-05-23-37-02 +0000,startribune,"In rare move, this northern Minnesota county i...",https://www.startribune.com/in-rare-move-this-...
50,30,2026-02-06-01-32-17 +0000,nypost,Nancy Guthrie’s neighbor reported suspicious w...,https://nypost.com/2026/02/05/us-news/nancy-gu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
